In [1]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 

In [2]:
# 准备数据
# 设置每个批次的数据量
batch_size = 64
# 将whc 转为 cwh
# 将数据标准化 均值和方差
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
# 加载数据
# 不进行打乱，不然特征和标签难以对应
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=batch_size)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [4]:
# 设计模型
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # minst的图片大小为28*28
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
 
    def forward(self, x):
        x = x.view(-1, 784)  # -1其实就是自动获取mini_batch
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)  # 最后一层不做激活，不进行非线性变换

In [5]:
model = Net()
# 构建损失和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [6]:
# 定义训练和测试过程
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))
 

In [7]:
 if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 2.146
[1,   600] loss: 0.743
[1,   900] loss: 0.398
accuracy on test set: 89 % 
[2,   300] loss: 0.313
[2,   600] loss: 0.267
[2,   900] loss: 0.231
accuracy on test set: 93 % 
[3,   300] loss: 0.190
[3,   600] loss: 0.175
[3,   900] loss: 0.159
accuracy on test set: 95 % 
[4,   300] loss: 0.133
[4,   600] loss: 0.130
[4,   900] loss: 0.121
accuracy on test set: 96 % 
[5,   300] loss: 0.101
[5,   600] loss: 0.101
[5,   900] loss: 0.097
accuracy on test set: 96 % 
[6,   300] loss: 0.079
[6,   600] loss: 0.079
[6,   900] loss: 0.078
accuracy on test set: 97 % 
[7,   300] loss: 0.063
[7,   600] loss: 0.064
[7,   900] loss: 0.063
accuracy on test set: 97 % 
[8,   300] loss: 0.051
[8,   600] loss: 0.051
[8,   900] loss: 0.051
accuracy on test set: 97 % 
[9,   300] loss: 0.041
[9,   600] loss: 0.041
[9,   900] loss: 0.041
accuracy on test set: 97 % 
[10,   300] loss: 0.032
[10,   600] loss: 0.033
[10,   900] loss: 0.032
accuracy on test set: 97 % 
